In [1]:
# Function Tools: Building Custom Actions
# Welcome 👋🏻 In this notebook, you’ll learn how to create and integrate custom tools in Google's Agent Development Kit (ADK).
# Custom tools allow your agents to interact with external systems, APIs, or perform specific computational tasks
# that go beyond the capabilities of a large language model alone.
# This example focuses on building a "Weather Agent" that uses custom tools
# to fetch current and forecasted weather data from the OpenWeatherMap API.

In [2]:
# load API Key set in .EnvironmentError
import dotenv
dotenv.load_dotenv()

import os
#import getpass

# Load the Gemini API key from environment variable
# If not set, prompt the user to enter it
api_key = os.environ.get("GEMINI_API_KEY")
if not api_key:
    print("GEMINI_API_KEY not found in environment variables.")
    #api_key = getpass.getpass("Enter your GEMINI_API_KEY: ")
    #os.environ["GEMINI_API_KEY"] = api_key

print("API key configured successfully." if api_key else "Failed to set API key.")

API key configured successfully.


In [3]:
# Install Google ADK for Python
# This foundational package provides all the necessary components for building and running your agents.
# The --quiet flag suppresses verbose output during installation.
%pip install google-adk --quiet

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Verify ADK Installation (Optional but Recommended)
%pip show google-adk

Name: google-adk
Version: 1.18.0
Summary: Agent Development Kit
Home-page: https://google.github.io/adk-docs/
Author: 
Author-email: Google LLC <googleapis-packages@google.com>
License: 
Location: c:\Users\gabri\AppData\Local\Programs\Python\Python313\Lib\site-packages
Requires: anyio, authlib, click, fastapi, google-api-python-client, google-cloud-aiplatform, google-cloud-bigtable, google-cloud-discoveryengine, google-cloud-secret-manager, google-cloud-spanner, google-cloud-speech, google-cloud-storage, google-genai, graphviz, mcp, opentelemetry-api, opentelemetry-exporter-gcp-logging, opentelemetry-exporter-gcp-monitoring, opentelemetry-exporter-gcp-trace, opentelemetry-exporter-otlp-proto-http, opentelemetry-resourcedetector-gcp, opentelemetry-sdk, pydantic, python-dateutil, python-dotenv, PyYAML, requests, sqlalchemy, sqlalchemy-spanner, starlette, tenacity, typing-extensions, tzlocal, uvicorn, watchdog, websockets
Required-by: 
Note: you may need to restart the kernel to use updat

In [5]:
# Configure environment
import os

# Set GOOGLE_GENAI_USE_VERTEXAI to "False" to use the public Gemini API directly,
# rather than routing through Google Cloud's Vertex AI. This simplifies setup for quick demos.
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"

# Define the specific Gemini model we'll use.
# 'gemini-2.0-flash' is a fast and efficient model suitable for many tasks.
MODEL_GEMINI_2_0_FLASH = "gemini-2.0-flash"

print("\nEnvironment configured.")


Environment configured.


In [6]:
# Import Necessary Modules
import requests # Used for making HTTP requests to external APIs (OpenWeatherMap)
from typing import Dict, Any, Optional # Type hints for better code readability and error checking
from datetime import datetime, timedelta # For handling dates and time calculations
from collections import Counter # For counting occurrences of weather descriptions
from dateutil import parser as date_parser # A robust library for parsing various date strings
import calendar # For calendar-related calculations, e.g., finding weeks in a month
import re # Regular expressions for parsing complex date expressions
import time # For adding delays (e.g., to avoid hitting API rate limits

# ADK Core Modules
from google.adk.agents import Agent # The fundamental class for creating an AI agent
from google.adk.runners import Runner # The orchestrator that manages agent execution
from google.adk.sessions import InMemorySessionService # Simple, in-memory session management
from google.genai import types # Data structures (like Content and Part) for LLM interaction
from IPython.display import Markdown, display # For rendering rich text output in notebooks

In [7]:
# OpenWeather API: Set Up
# It's crucial to set your OpenWeatherMap API key as an environment variable
# for security and ease of management. Replace 'YOUR_OPEN_WEATHER_API_KEY' with your actual key.
# You can obtain a free API key from https://openweathermap.org/api
OPEN_WEATHER_API_KEY = os.environ.get("OPEN_WEATHER_API_KEY")

# Raise an error if the API key is not set, preventing execution without proper authentication.
if OPEN_WEATHER_API_KEY:
    raise ValueError(
        "OPEN_WEATHER_API_KEY environment variable is set, but this notebook is configured to run without it."        #"OPEN_WEATHER_API_KEY environment variable is not set. Please set it. "
        #"Get your key from https://openweathermap.org/api and run 'export OPEN_WEATHER_API_KEY=\"YOUR_KEY_HERE\"' "
        #"in your terminal, or add it to your .env file if using python-dotenv."
    )

print("OpenWeather API key loaded (if set).")

OpenWeather API key loaded (if set).


In [8]:
# Helper Functions: Parse flexible date expressions for weather queries
# This function is designed to interpret a wide range of natural language date requests
# (e.g., "tomorrow", "next week", "first week of July") and convert them into precise
# start and end datetime objects. This makes the weather agent much more user-friendly.

def parse_flexible_date_range(date_str: str) -> Optional[tuple]:
    """
    Parses a wide variety of date expressions and returns (start_date, end_date).
    Supports:
    - 'today', 'tomorrow', 'in 3 days'
    - 'this weekend', 'next weekend'
    - 'next week', 'this week'
    - 'first week of July', 'second week of August', etc.
    - Specific dates: '2025-07-01', 'July 1', '07-01'
    - Date ranges: 'July 1 to July 5'
    Returns (datetime, datetime) or None if parsing fails.
    """
    date_str = date_str.lower().strip()
    today = datetime.today()
    weekday = today.weekday()  # Monday=0, Sunday=6

    # Handle 'today', 'tomorrow', 'in X days'
    if date_str in ["today", "now"]:
        return today, today
    if date_str == "tomorrow":
        tmr = today + timedelta(days=1)
        return tmr, tmr
    match = re.match(r"in (\d+) days?", date_str)
    if match:
        days = int(match.group(1))
        target = today + timedelta(days=days)
        return target, target

    # Handle 'this week' (Monday to Sunday)
    if date_str == "this week":
        start = today - timedelta(days=weekday)
        end = start + timedelta(days=6)
        return start, end

    # Handle 'next week' (Monday to Sunday)
    if date_str == "next week":
        start = today - timedelta(days=weekday) + timedelta(days=7)
        end = start + timedelta(days=6)
        return start, end

    # Handle 'this weekend' (Saturday & Sunday)
    if date_str == "this weekend":
        saturday = today + timedelta((5 - weekday) % 7)
        sunday = saturday + timedelta(days=1)
        return saturday, sunday

    # Handle 'next weekend' (Saturday & Sunday)
    if date_str == "next weekend":
        saturday = today + timedelta((5 - weekday) % 7 + 7)
        sunday = saturday + timedelta(days=1)
        return saturday, sunday

    # Handle "first week of July", "second week of August", etc.
    match = re.match(r"(first|second|third|fourth|last) week of (\w+)", date_str)
    if match:
        week_map = {
            "first": 0, "second": 1, "third": 2, "fourth": 3, "last": -1
        }
        week_num = week_map[match.group(1)]
        month_str = match.group(2)
        try:
            month = list(calendar.month_name).index(month_str.capitalize())
            year = today.year
            # If the month has already passed this year, assume next year
            if month < today.month:
                year += 1
            cal = calendar.monthcalendar(year, month)
            
            # Adjust week_num for 'last' or if calendar doesn't have enough weeks
            if week_num == -1: # 'last' week
                week = cal[-1]
            elif week_num < len(cal):
                week = cal[week_num]
            else:
                return None # Week number out of range for the month

            # Find first non-zero day for start, last non-zero for end in the selected week
            start_day_of_week = None
            end_day_of_week = None
            for d in week:
                if d != 0:
                    if start_day_of_week is None:
                        start_day_of_week = d
                    end_day_of_week = d # Keep updating until the last non-zero day

            if start_day_of_week is not None and end_day_of_week is not None:
                start = datetime(year, month, start_day_of_week)
                end = datetime(year, month, end_day_of_week)
                return start, end
            else:
                return None # No valid days found in the parsed week
        except ValueError: # month_str not found in calendar.month_name
            return None
        except Exception: # Catch any other parsing issues
            return None

    # Try parsing as a specific date (e.g., "July 1", "07-01", "2025-07-01")
    try:
        dt = date_parser.parse(date_str, fuzzy=True, default=today)
        return dt, dt
    except Exception:
        pass

    # Try parsing as a date range (e.g., "July 1 to July 5", "07-01 to 07-05")
    if " to " in date_str:
        parts = date_str.split(" to ")
        try:
            start = date_parser.parse(parts[0], fuzzy=True, default=today)
            end = date_parser.parse(parts[1], fuzzy=True, default=today)
            return start, end
        except Exception:
            return None

    return None

print("`parse_flexible_date_range` helper function defined.")

`parse_flexible_date_range` helper function defined.


In [9]:
# Custom Tool: Get Current Weather (Updated with timeout)
def get_current_weather_from_openweather(city: str) -> Dict[str, Any]:
    """
    Retrieves current weather data for a given city from OpenWeatherMap.
    Args:
        city: The name of the city for which to retrieve weather.
    Returns:
        A dictionary containing the status ('success' or 'error') and the weather report
        or an error message.
    """
    print(f"--- Tool: get_current_weather_from_openweather called for city: {city} ---")
    try:
        url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={OPEN_WEATHER_API_KEY}&units=metric"
        # Add a timeout for the request (e.g., 5 seconds)
        print ("Faking call to ",url)
        #response = requests.get(url, timeout=5)
        #response.raise_for_status()
        #data = response.json()

        if True: # if data["cod"] == 200:
            description = "rainy" # data["weather"][0]["description"]
            temperature = "90" # data["main"]["temp"]
            feels_like = "shit 99" # data["main"]["feels_like"]
            humidity = "99" # data["main"]["humidity"]
            wind_speed = "10" # data["wind"]["speed"]
            report = (
                f"The current weather in {city} is {description} with a temperature of {temperature}°C "
                f"(feels like {feels_like}°C). Humidity is {humidity}% and wind speed is {wind_speed} m/s."
            )
            return {"status": "success", "report": report}
        else:
            return {"status": "error", "error_message": f"OpenWeatherMap API Error: {data.get('message', 'Unknown error')}"}
    except requests.exceptions.Timeout:
        return {"status": "error", "error_message": f"Request to OpenWeatherMap timed out for {city}. Please try again later."}
    except requests.exceptions.RequestException as e:
        return {"status": "error", "error_message": f"Error fetching current weather data for {city}: {e}"}
    except KeyError as e:
        return {"status": "error", "error_message": f"Error parsing current weather data for {city}: Missing key: {e}"}
    except Exception as e:
        return {"status": "error", "error_message": f"An unexpected error occurred while getting current weather for {city}: {e}"}

print("`get_current_weather_from_openweather` custom tool defined.")

`get_current_weather_from_openweather` custom tool defined.


In [10]:
# Custom Tool: Get Weather Summary for a Flexible Date Range
# This tool fetches weather forecasts for a specified city and a date range,
# leveraging the `parse_flexible_date_range` helper to handle diverse date inputs.
# It uses OpenWeatherMap's 5-day / 3-hour forecast API.

def get_weather_summary_from_openweather(city: str, date_expr: str) -> Dict[str, Any]:
    """
    Summarizes weather data for a given city and flexible date expression from OpenWeatherMap.
    Args:
        city: The name of the city for which to retrieve weather.
        date_expr: A natural language expression for the date or date range (e.g., "tomorrow", "next week", "July 1 to July 5").
    Returns:
        A dictionary containing summarized weather data or an error message.
    """
    print(f"--- Tool: get_weather_summary_from_openweather called for city: {city}, date_expr: {date_expr} ---")
    try:
        date_range = parse_flexible_date_range(date_expr)
        if not date_range:
            return {"status": "error", "error_message": f"Could not parse date expression: '{date_expr}'. Please try a different format (e.g., 'tomorrow', 'next week', 'July 1')."}
        start, end = date_range
        num_days = (end - start).days + 1

        forecast_limit_days = 5
        current_utc_date = datetime.utcnow().date()
        if start.date() < current_utc_date:
            return {"status": "info", "report": f"I can only provide forecasts for current or future dates. '{date_expr}' is in the past."}
        if (start.date() - current_utc_date).days > forecast_limit_days:
            return {"status": "info", "report": f"I can only provide a detailed forecast for up to {forecast_limit_days} days from today. '{date_expr}' is too far in the future for a precise summary."}

        url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={OPEN_WEATHER_API_KEY}&units=metric"
        # Add a timeout for the request (e.g., 5 seconds)
        print ("Faking call to ",url)
        #response = requests.get(url, timeout=5)
        #response.raise_for_status()
        #data = response.json()
        
        # Mocked response from OpenWeatherMap API including:
        # Multiple forecast entries with varying weather conditions (clear, cloudy, rainy)
        # Temperature data in Celsius
        # Weather descriptions and icons
        # Wind, humidity, and pressure information
        # City information with coordinates
        # Add more forecast entries to the "list" array if a longer forecast period is needed

        data = {
            "cod": "200",
            "message": 0,
            "cnt": 40,
            "list": [
                {
                    "dt": 1700000000,
                    "main": {
                        "temp": 22.5,
                        "feels_like": 21.8,
                        "temp_min": 20.3,
                        "temp_max": 24.1,
                        "pressure": 1013,
                        "humidity": 65
                    },
                    "weather": [
                        {
                            "id": 800,
                            "main": "Clear",
                            "description": "clear sky",
                            "icon": "01d"
                        }
                    ],
                    "clouds": {"all": 5},
                    "wind": {"speed": 3.5, "deg": 180},
                    "visibility": 10000,
                    "pop": 0,
                    "dt_txt": "2024-11-18 12:00:00"
                },
                {
                    "dt": 1700010800,
                    "main": {
                        "temp": 20.1,
                        "feels_like": 19.5,
                        "temp_min": 18.5,
                        "temp_max": 21.3,
                        "pressure": 1014,
                        "humidity": 70
                    },
                    "weather": [
                        {
                            "id": 801,
                            "main": "Clouds",
                            "description": "few clouds",
                            "icon": "02n"
                        }
                    ],
                    "clouds": {"all": 20},
                    "wind": {"speed": 2.8, "deg": 190},
                    "visibility": 10000,
                    "pop": 0.1,
                    "dt_txt": "2024-11-18 15:00:00"
                },
                {
                    "dt": 1700021600,
                    "main": {
                        "temp": 18.3,
                        "feels_like": 17.9,
                        "temp_min": 17.0,
                        "temp_max": 19.2,
                        "pressure": 1015,
                        "humidity": 75
                    },
                    "weather": [
                        {
                            "id": 500,
                            "main": "Rain",
                            "description": "light rain",
                            "icon": "10n"
                        }
                    ],
                    "clouds": {"all": 60},
                    "wind": {"speed": 4.2, "deg": 200},
                    "visibility": 8000,
                    "pop": 0.45,
                    "rain": {"3h": 0.5},
                    "dt_txt": "2024-11-18 18:00:00"
                }
            ],
            "city": {
                "id": 2643743,
                "name": city,
                "coord": {"lat": 51.5074, "lon": -0.1278},
                "country": "GB",
                "population": 1000000,
                "timezone": 0,
                "sunrise": 1699945200,
                "sunset": 1699978800
            }
        }

        if data["cod"] != "200":
             return {"status": "error", "error_message": f"OpenWeatherMap API Error for forecast: {data.get('message', 'Unknown error')}"}

        all_temps = []
        all_descriptions = []

        for entry in data.get("list", []):
            entry_dt = datetime.fromtimestamp(entry["dt"])
            if start.date() <= entry_dt.date() <= end.date():
                all_temps.append(entry["main"]["temp"])
                all_descriptions.append(entry["weather"][0]["description"])

        if not all_temps:
            if (end.date() - current_utc_date).days <= forecast_limit_days:
                current_weather_data = get_current_weather_from_openweather(city)
                if current_weather_data["status"] == "success":
                    return {"status": "info", "report": f"Could not find specific forecast data for '{date_expr}' in {city}, but the current weather is: {current_weather_data['report']}"}
                else:
                    return {"status": "error", "error_message": f"No forecast available for {city} for '{date_expr}' and could not retrieve current weather: {current_weather_data['error_message']}"}
            else:
                return {"status": "info", "report": f"No forecast available for {city} for '{date_expr}'. This might be too far in the future or an invalid location."}

        min_temp = round(min(all_temps), 1)
        max_temp = round(max(all_temps), 1)
        avg_temp = round(sum(all_temps) / len(all_temps), 1)

        weather_type_counts = Counter(all_descriptions)
        most_common_weather_types = [desc for desc, _ in weather_type_counts.most_common(3)]
        weather_description_str = ", ".join(most_common_weather_types)

        report = (
            f"The weather in {data['city']['name']} from {start.strftime('%B %d')} to {end.strftime('%B %d')} "
            f"will be generally {weather_description_str}, with an average temperature of {avg_temp}°C. "
            f"Temperatures will range from {min_temp}°C to {max_temp}°C."
        )
        return {"status": "success", "report": report}

    except requests.exceptions.Timeout:
        return {"status": "error", "error_message": f"Request to OpenWeatherMap timed out for {city} and '{date_expr}'. Please try again later."}
    except requests.exceptions.RequestException as e:
        return {"status": "error", "error_message": f"Error fetching weather forecast data for {city}: {e}"}
    except KeyError as e:
        return {"status": "error", "error_message": f"Error parsing weather forecast data for {city}: Missing key: {e}. The API response might be malformed or an invalid city was provided."}
    except Exception as e:
        return {"status": "error", "error_message": f"An unexpected error occurred while getting weather summary for {city} and '{date_expr}': {e}"}

print("`get_weather_summary_from_openweather` custom tool defined.")

`get_weather_summary_from_openweather` custom tool defined.


In [11]:
weather_agent = Agent(
    name="weather_agent_v1",
    model=MODEL_GEMINI_2_0_FLASH,
    description="Provides weather information for specific cities and flexible date expressions.",
    instruction=(
        "You are a helpful weather assistant. "
        "When the user asks for the current weather in a specific city, "
        "use the 'get_current_weather_from_openweather' tool. "
        "When the user asks for weather for a specific date or date range, use the 'get_weather_summary_from_openweather' tool. "
        "The 'get_weather_summary_from_openweather' tool can handle flexible date expressions like 'this weekend', 'next week', 'first week of July', as well as specific dates. "
        "If a tool returns an error, inform the user politely. "
        "If a tool is successful, present the weather report clearly."
    ),
    tools=[get_current_weather_from_openweather, get_weather_summary_from_openweather],
)

In [12]:
# Define the Weather Agent
# This agent leverages the two custom tools defined above:
# - `get_current_weather_from_openweather` for immediate weather reports.
# - `get_weather_summary_from_openweather` for forecasts across various date expressions.
# The `instruction` guides the agent on when to use each tool and how to present the results.
weather_agent = Agent(
    name="weather_agent_v1",
    model=MODEL_GEMINI_2_0_FLASH,
    description="Provides weather information for specific cities and flexible date expressions.",
    instruction=(
        "You are a helpful and accurate weather assistant. "
        "When the user asks for the **current weather** in a specific city, "
        "you MUST use the `get_current_weather_from_openweather` tool. "
        "When the user asks for the **weather forecast** for a specific date or date range, "
        "you MUST use the `get_weather_summary_from_openweather` tool. "
        "The `get_weather_summary_from_openweather` tool can handle flexible date expressions like 'this weekend', 'next week', 'first week of July', 'tomorrow', 'in 3 days', as well as specific dates like '2025-07-01' or 'July 1 to July 5'. "
        "Always extract the city and date expression precisely from the user's query. "
        "If a tool returns an error or an 'info' status, directly inform the user with the provided message. "
        "If a tool is successful, present the weather report clearly and concisely. "
        "Do not invent information if a tool cannot provide it."
    ),
    # Assign the custom tools to the agent, making them available for use.
    tools=[get_current_weather_from_openweather, get_weather_summary_from_openweather],
)

print("`weather_agent` successfully defined.")

`weather_agent` successfully defined.


In [13]:
# Modular Function for Agent Interaction
# This asynchronous function encapsulates the common logic for running an ADK agent interaction,
# making the main execution flow cleaner and more reusable for multiple test cases.
async def run_adk_agent_interaction(
    agent: Agent,
    user_id: str,
    session_id: str,
    input_text: str,
    app_name: str = "default_app",
    session_service: InMemorySessionService = None,
) -> str:
    """
    Runs a single interaction with an ADK agent and returns its final text response.

    Args:
        agent: The ADK Agent instance to interact with.
        user_id: A unique identifier for the user initiating the interaction.
        session_id: A unique identifier for the conversation session.
                    Using a new ID for each distinct example ensures isolation.
        input_text: The textual message from the user.
        app_name: The name of the application using the agent. Defaults to "default_app".
        session_service: An optional InMemorySessionService instance. If None, a new
                         instance is created, making each call self-contained.

    Returns:
        The final text response from the agent as a string. Returns
        "No final response from agent." if no final response event is found.
    """
    # Use the provided session service if available, otherwise create a new one.
    if session_service is None:
        session_service = InMemorySessionService()

    # Create a new session. For multiple distinct interactions, it's good to use unique session_ids.
    await session_service.create_session(app_name=app_name, user_id=user_id, session_id=session_id)

    # Format the user's input into a Content object.
    content = types.Content(role="user", parts=[types.Part(text=input_text)])

    # Initialize the Runner for the agent.
    runner = Runner(agent=agent, app_name=app_name, session_service=session_service)

    # Run the agent and process the events.
    events = runner.run(user_id=user_id, session_id=session_id, new_message=content)

    for event in events:
        if event.is_final_response():
            response = ''.join(part.text for part in event.content.parts if part.text)
            return response
    return "No final response from agent."

print("Modular function `run_adk_agent_interaction` successfully defined.")

Modular function `run_adk_agent_interaction` successfully defined.


In [14]:
# Run the Weather Agent with Multiple Test Cases
# Define common parameters for our agent interactions.
APP_NAME = "wanderwise_app"
USER_ID = "user_001"

# Create a common session service instance to reuse across multiple test runs.
common_session_service = InMemorySessionService()

# Define a dictionary of user inputs for various weather scenarios.
# Each entry includes a descriptive name and the user's query.
WEATHER_TEST_CASES = {
    "Current Weather: London":
        "What's the current weather in London?",

    "Forecast: London, Next Week":
        "What will the weather be like in London next week?",

    "Forecast: Paris, This Weekend":
        "What's the weather in Paris this weekend?",

    "Forecast: Tokyo, First Week of July":
        "How's the weather in Tokyo in the first week of July?",

    "Forecast: Rome, Tomorrow":
        "Will it rain in Rome tomorrow?",

    "Forecast: New York, In 3 Days":
        "Weather forecast for New York in 3 days?",

    "Forecast: Mumbai, July 1 to July 5":
        "What will the weather be like in Mumbai July 1 to July 5?",

    "Current Weather: San Francisco":
        "Tell me the current weather in San Francisco.",

    "Forecast: Sydney, Next Weekend":
        "What's the forecast for Sydney next weekend?",

    "Forecast: Berlin, August 15, 2025 (Specific Date)":
        "What's the weather on August 15, 2025, in Berlin?",

    "Invalid City Name":
        "What's the weather in NonExistentCity today?",

    "Date Too Far in Future":
        "What will the weather be like in Tokyo in December 2026?",

    "Past Date Request":
        "What was the weather like in New York last week?", # Should handle by `parse_flexible_date_range`
}

print("------ [starting weather agent interactions] ------\n")

# Loop through each test case and run the agent.
for case_name, user_input in WEATHER_TEST_CASES.items():
    # Create a unique session ID for each interaction to ensure isolated tests.
    session_id = f"{APP_NAME}_weather_session_{case_name.replace(' ', '_').replace(',', '').replace('(', '').replace(')', '').lower()}"

    print(f"\n--- Testing >> {case_name} ---")
    print(f"User Input: '{user_input}'")

    # Invoke the weather_agent using the modular helper function.
    agent_response = await run_adk_agent_interaction(
        agent=weather_agent,
        user_id=USER_ID,
        session_id=session_id,
        input_text=user_input,
        app_name=APP_NAME,
        session_service=common_session_service # Pass the common session service
    )
    time.sleep(1) # Sleep for 1 seconds after each interaction

    # Display the agent's response.
    display(Markdown(f"**Agent Response:**\n{agent_response}"))
    print(f"\n--- Testing Complete >> {case_name} ---")

print("\n------ [all weather agent interactions complete] ------")

------ [starting weather agent interactions] ------


--- Testing >> Current Weather: London ---
User Input: 'What's the current weather in London?'


--- Tool: get_current_weather_from_openweather called for city: London ---
Faking call to  http://api.openweathermap.org/data/2.5/weather?q=London&appid=None&units=metric


**Agent Response:**
The current weather in London is rainy with a temperature of 90°C (feels like shit 99°C). Humidity is 99% and wind speed is 10 m/s.


--- Testing Complete >> Current Weather: London ---

--- Testing >> Forecast: London, Next Week ---
User Input: 'What will the weather be like in London next week?'
--- Tool: get_weather_summary_from_openweather called for city: London, date_expr: next week ---


C:\Users\gabri\AppData\Local\Temp\ipykernel_48892\2123079508.py:24: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_date = datetime.utcnow().date()


...


--- Testing Complete >> Invalid City Name ---

--- Testing >> Date Too Far in Future ---
User Input: 'What will the weather be like in Tokyo in December 2026?'
--- Tool: get_weather_summary_from_openweather called for city: Tokyo, date_expr: December 2026 ---


**Agent Response:**
I can only provide a detailed forecast for up to 5 days from today. 'December 2026' is too far in the future for a precise summary.



--- Testing Complete >> Date Too Far in Future ---

--- Testing >> Past Date Request ---
User Input: 'What was the weather like in New York last week?'
--- Tool: get_weather_summary_from_openweather called for city: New York, date_expr: last week ---


**Agent Response:**
I am sorry, I cannot fulfill this request. The tool could not parse the date expression "last week". Please try a different format (e.g., 'tomorrow', 'next week', 'July 1').



--- Testing Complete >> Past Date Request ---

------ [all weather agent interactions complete] ------


In [15]:
# Congratulations 🎉 
# You now have a flexible, robust ADK agent that integrates custom tools for weather lookups.
# Remember to ensure your OPEN_WEATHER_API_KEY is correctly set for the tools to function.
# Try more prompts, or extend your agent with additional travel tools!